# Objective of this pipeline:


*   Read the dataset of PGJ CDMX crimes
*   Print the crimes in a folium map
*   Extract the info about origin and destiny from a google sheets
*   Convert the address to coordenates
*   Draw the safest and shortest routes from an origin to a destiny in CDMX





Install libraries

In [ ]:
!pip install geocoder
!pip install fontawesome
!pip install pyproj
!pip install openrouteservice

#Converting address to coordinates
!pip install geopandas
!pip install geopy

     |████████████████████████████████| 102kB 5.7MB/s 
     |████████████████████████████████| 6.5MB 4.3MB/s 
     |████████████████████████████████| 1.0MB 7.6MB/s 
     |████████████████████████████████| 14.8MB 324kB/s 


Import libraries

In [ ]:
#import libraries for data wrangling
import pandas as pd 
import numpy as np

#import libraries for geo-visualization
import folium
from folium import plugins
from folium.plugins import HeatMap
import ipywidgets

#import geocoder
from vega_datasets import data as vds
from branca.element import Figure

#import fontawesome as fa

#import libraries for openrouteservice 
import json
import requests
import pyproj
from shapely import geometry
from shapely.geometry import Point, LineString, Polygon, MultiPolygon
from openrouteservice import client

#connect to google sheets
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials

#geopy
from geopy.geocoders import Nominatim

Connection to drive

In [ ]:
#Google Drive connection
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset: victimas_completa_con_semaforo_base_pgj.csv

In [ ]:
#Read dataset
acoso_fem = pd.read_csv('/content/drive/MyDrive/Bases Collab/DB_con_200_sample_segun_escala_clasificacion.csv')
acoso_fem.shape

(200, 21)

In [ ]:
#experimentaremos los nodos para ver si es un tema de velocidad
acoso_fem = acoso_fem[0:150]

In [ ]:
acoso_fem.head(1)

,idCarpeta,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,competencia,Año_hecho,Mes_hecho,AlcaldiaHechos,ColoniaHechos,Calle_hechos,Calle_hechos2,latitud,longitud,Hora_Hechos_24,Tipo_de_delito,Años_sentencia,Escala_clasificacion,Casos
0,8380337,ROBO A NEGOCIO CON VIOLENCIA,ROBO A NEGOCIO CON VIOLENCIA,NaN,NaN,MORAL,VICTIMA,FUERO COMUN,2019,Febrero,Miguel Hidalgo,SAN JUANICO,LAGO CHIEM,LAGO WAM,19.45108,-99.19053,9,ROBO A NEGOCIO CON VIOLENCIA,3,1,1


In [ ]:
#Replace np.nan with the location of mexico city
acoso_fem['latitud'] = acoso_fem['latitud'].fillna(19.4978)
acoso_fem['longitud'] = acoso_fem['longitud'].fillna(-99.1269)

In [ ]:
#revisamos los nulos dentro de la base no existan nulos para las geolocalizaciones ni tampoco para los tiempos en 24 horas
#acoso_fem.isnull().sum()

#MAPA POR TEMPORALIDAD

In [ ]:
#CRITERIO DE CLASIFICACIÓN 
#21pm a 4am noche 
#5am a 12pm mañana 
#13pm a 20pm tarde 

#Create a copy of the dataframe 
acoso_fem_2 = acoso_fem.copy()

#Create a list of our conditions 
conditions = [
              (acoso_fem_2['Hora_Hechos_24'] > 4) & (acoso_fem_2['Hora_Hechos_24'] < 13),
              (acoso_fem_2['Hora_Hechos_24'] > 12) & (acoso_fem_2['Hora_Hechos_24'] < 21)
              ]

#Create a list of the values we want to assign for each condition 
values = ['mañana', 'tarde']

#Create a new column and use np.select to asign values to is using our conditions
acoso_fem_2['Momento'] = np.select(conditions, values)

#Replace zeros for 'noche' Dataframe 
acoso_fem_2['Momento'] = acoso_fem_2['Momento'].replace({'0':'noche'})

#Display updated Dataframe 
acoso_fem_2.head(1)

,idCarpeta,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,competencia,Año_hecho,Mes_hecho,AlcaldiaHechos,ColoniaHechos,Calle_hechos,Calle_hechos2,latitud,longitud,Hora_Hechos_24,Tipo_de_delito,Años_sentencia,Escala_clasificacion,Casos,Momento
0,8380337,ROBO A NEGOCIO CON VIOLENCIA,ROBO A NEGOCIO CON VIOLENCIA,NaN,NaN,MORAL,VICTIMA,FUERO COMUN,2019,Febrero,Miguel Hidalgo,SAN JUANICO,LAGO CHIEM,LAGO WAM,19.45108,-99.19053,9,ROBO A NEGOCIO CON VIOLENCIA,3,1,1,mañana


In [ ]:
#Creamos 3 diferentes dataframes donde cada uno contenga los momentos del día
Momento1 = acoso_fem_2[acoso_fem_2['Momento'] == 'mañana']
Momento2 = acoso_fem_2[acoso_fem_2['Momento'] == 'tarde']
Momento3 = acoso_fem_2[acoso_fem_2['Momento'] == 'noche']

In [ ]:
#Ahora convertimos cada uno de esos dataframes a un array por cada momento

#create an array of latitud and longitud
array_momento1 = np.array(Momento1[['latitud','longitud']])
array_momento2 = np.array(Momento2[['latitud','longitud']])
array_momento3 = np.array(Momento3[['latitud','longitud']])

In [ ]:
#CreateBufferPolygon function
#the radius is the radius of the yellow buffer
def CreateBufferPolygon(point_in, resolution=10, radius=800):    

    sr_wgs = pyproj.Proj(init='epsg:4326')
    sr_utm = pyproj.Proj(init='epsg:32632') # WGS84 UTM32N
    point_in_proj = pyproj.transform(sr_wgs, sr_utm, *point_in) # unpack list to arguments
    point_buffer_proj = Point(point_in_proj).buffer(radius, resolution=resolution) # 50 m buffer
    
    # Iterate over all points in buffer and build polygon
    poly_wgs = []
    for point in point_buffer_proj.exterior.coords:
        poly_wgs.append(pyproj.transform(sr_utm, sr_wgs, *point)) # Transform back to WGS84

    return poly_wgs

In [ ]:
# Set up the fundamentals
api_key = '5b3ce3597851110001cf6248b5f681c8c8a84d67afcdffa50f7316ca' # Individual api key
clnt = client.Client(key=api_key) # Create client with api key

In [ ]:
#creamos el mapa
map_params_nuestro = {'tiles':'Stamen Toner',
              'location':([19.4978, -99.1269]),
              'zoom_start': 12}
fg = folium.FeatureGroup(name="Momento_día")
map1_nuestro = folium.Map(**map_params_nuestro).add_child(fg)

#PLUGIN PARA AGREGAR FILTRO DE SUBCATEGORIES

g1 = plugins.FeatureGroupSubGroup(fg, "Mañana")
map1_nuestro.add_child(g1)

g2 = plugins.FeatureGroupSubGroup(fg, "Tarde")
map1_nuestro.add_child(g2)

g3 = plugins.FeatureGroupSubGroup(fg, "Noche")
map1_nuestro.add_child(g3)



folium.LayerControl(collapsed=False).add_to(map1_nuestro)


In [ ]:
# Populate a crime site buffer polygon list
sites_poly_nuestro = []
x = []
for coordenada in array_momento1:
  site_coords_nuestro = coordenada
  site_coords_cr1 = list(site_coords_nuestro)
  folium.features.Marker(site_coords_cr1,
  popup='Mañana <br>{0}'.format(site_coords_cr1),
  icon=folium.Icon(color="lightblue")).add_to(map1_nuestro).add_to(g1)

  
    
  # Create buffer polygons around construction sites with 10 m radius and low resolution
  site_poly_coords_nuestro1 = CreateBufferPolygon(reversed(site_coords_cr1),
                                         resolution=2, # low resolution to keep polygons lean
                                           radius=1000)
  sites_poly_nuestro.append(site_poly_coords_nuestro1)
    
  site_poly_coords_nuestro1 = [(y,x) for x,y in site_poly_coords_nuestro1] # Reverse coords for folium/Leaflet
  folium.vector_layers.Polygon(locations=site_poly_coords_nuestro1,
                               color='#ffd699',
                               fill_color='#ffd699',
                               fill_opacity=0.2,
                               weight=3).add_to(map1_nuestro).add_to(g1)


for coordenada in array_momento2:
  site_coords_nuestro2 = coordenada
  site_coords_cr2 = list(site_coords_nuestro2)
  folium.features.Marker(site_coords_cr2,
  popup='Tarde <br>{0}'.format(site_coords_cr2),
  icon=folium.Icon(color="blue")).add_to(map1_nuestro).add_to(g2)

   # Create buffer polygons around construction sites with 10 m radius and low resolution
  site_poly_coords_nuestro2 = CreateBufferPolygon(reversed(site_coords_cr2),
                                         resolution=2, # low resolution to keep polygons lean
                                           radius=1000)
  sites_poly_nuestro.append(site_poly_coords_nuestro2)
    
  site_poly_coords_nuestro2 = [(y,x) for x,y in site_poly_coords_nuestro2] # Reverse coords for folium/Leaflet
  folium.vector_layers.Polygon(locations=site_poly_coords_nuestro2,
                               color='#ffd699',
                               fill_color='#ffd699',
                               fill_opacity=0.2,
                               weight=3).add_to(map1_nuestro).add_to(g2)

for coordenada in array_momento3:
  site_coords_nuestro3 = coordenada
  site_coords_cr3 = list(site_coords_nuestro3)
  folium.features.Marker(site_coords_cr3,
  popup='Noche <br>{0}'.format(site_coords_cr3),
  icon=folium.Icon(color="darkblue")).add_to(map1_nuestro).add_to(g3)

   # Create buffer polygons around construction sites with 10 m radius and low resolution
  site_poly_coords_nuestro3 = CreateBufferPolygon(reversed(site_coords_cr3),
                                         resolution=2, # low resolution to keep polygons lean
                                           radius=1000)
  sites_poly_nuestro.append(site_poly_coords_nuestro3)
    
  site_poly_coords_nuestro3 = [(y,x) for x,y in site_poly_coords_nuestro3] # Reverse coords for folium/Leaflet
  folium.vector_layers.Polygon(locations=site_poly_coords_nuestro3,
                               color='#ffd699',
                               fill_color='#ffd699',
                               fill_opacity=0.2,
                               weight=3).add_to(map1_nuestro).add_to(g3)




/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axi

In [ ]:
map1_nuestro

# EXTRAER INFO DE GOOGLE SHEETS Y CONVERTIR DIRECCIÓN A COORDENADAS

In [ ]:
#authenticate user
auth.authenticate_user()

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('prueba folium').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
tabla = pd.DataFrame.from_records(rows)
tabla

forma_transporte = tabla[1][2]
geolocator = Nominatim(user_agent="example app")

#find the location of the address in column 1
tabla["loc"] = tabla[1].apply(geolocator.geocode)

#create a new column called "loc" where latitud,longitud and altitude will be registered
tabla["point"]= tabla["loc"].apply(lambda loc: tuple(loc.point) if loc else None)
#print(tabla)

#create the columns 'lat','lon','altitude
tabla[['lat', 'lon', 'altitude']] = pd.DataFrame(tabla['point'].to_list(), index=tabla.index)
tabla

#create variables for each coordenate and save them into a variable
lon_inicio = tabla['lon'][0]
lon_fin = tabla['lon'][1]

lat_inicio = tabla['lat'][0]
lat_fin = tabla['lat'][1]


print(f'longitud inicial: {lon_inicio}')
print(f'latitud inicial: {lat_inicio}')
print(f'longitud final: {lon_fin}')
print(f'latitud final: {lat_fin}')
print(f'forma transporte: {forma_transporte}')

longitud inicial: -99.1184576
latitud inicial: 19.4816347
longitud final: -99.16211723484005
latitud final: 19.42885515
forma transporte: driving-car


In [ ]:
# GeoJSON style function
def style_function(color):
    return lambda feature: dict(color=color,
                              weight=3,
                              opacity=0.5)

# Create new map to start from scratch
map_params_nuestro.update({'location': ([19.426750249999998, -99.14928202618324]), #Metro Balderas
                   'zoom_start': 12})
map2_nuestro = folium.Map(**map_params_nuestro)

# Request normal route between appropriate locations without construction sites
request_params_nuestro = {'coordinates': [[lon_inicio, lat_inicio], #canal de cuemanco (-99.1023365, 19.2774178) (Liverpool polanco -99.18306062075595, 19.4346181)
                                 [lon_fin, lat_fin]], #la villa(-99.1184576, 19.4816347) (Escuela libre de derecho -99.14581995,19.42593495)
                'format_out': 'geojson',
                'profile': forma_transporte, #cycling-regular , driving-car, foot-walking
                'preference': 'shortest',
                'instructions': 'false',}

route_normal_nuestro = clnt.directions(**request_params_nuestro)
folium.features.GeoJson(data=route_normal_nuestro,
                        name='Shortest route',
                        style_function=style_function('#FF0000'),
                        overlay=True).add_to(map2_nuestro)

# Buffer route with 0.009 degrees (really, just too lazy to project again...)
#This buffer "limits" the number of crimes it will be consider
x = LineString(request_params_nuestro['coordinates'][0:2]).buffer(0.05) #investigar el buffer en que unidad esta (millas, metros) (0.009)
route_buffer_nuestro = x
folium.features.GeoJson(data=geometry.mapping(route_buffer_nuestro),
                        name='Route Buffer',
                        style_function=style_function('#FFFF00'),
                        overlay=True).add_to(map2_nuestro)

#LineString(route_normal['features'][0]['geometry']['coordinates']).buffer(0.009)

# Plot which construction sites fall into the buffer Polygon
sites_buffer_poly_nuestro = []
for site_poly in sites_poly_nuestro:
    poly_nuestro = Polygon(site_poly)
    if route_buffer_nuestro.intersects(poly_nuestro):
        folium.features.Marker(list(reversed(poly_nuestro.centroid.coords[0]))).add_to(map2_nuestro)
        sites_buffer_poly_nuestro.append(poly_nuestro)

In [ ]:
map2_nuestro

In [ ]:
# Add the site polygons to the request parameters
#try:
request_params_nuestro['options'] = {'avoid_polygons': geometry.mapping(MultiPolygon(sites_buffer_poly_nuestro))}
route_detour_nuestro = clnt.directions(**request_params_nuestro)

folium.features.GeoJson(data=route_detour_nuestro,
                        name='Route avoiding crime sites',
                        style_function=style_function('#00FF00'),
                        overlay=True).add_to(map2_nuestro)

map2_nuestro.add_child(folium.map.LayerControl())

#except:
  #print('Lo sentimos pero no encontramos una alternativa de ruta más segura :(')


In [ ]:
safest_duration = 0
shortest_duration = 0

safest_distance = 0
shortest_distance = 0

safest_duration = round((route_detour_nuestro['features'][0]['properties']['summary']['duration'] / 60),2)
shortest_duration = round((route_normal_nuestro['features'][0]['properties']['summary']['duration'] / 60),2)
safest_distance = round((route_detour_nuestro['features'][0]['properties']['summary']['distance'] / 1000),2)
shortest_distance = round((route_normal_nuestro ['features'][0]['properties']['summary']['distance'] / 1000),2)

print(f'Safest route: {safest_duration} mins & {safest_distance}km')
print('')
print(f'Shortest route: {shortest_duration} mins & {shortest_distance}km')                                                                                                           

Safest route: 24.92 mins & 9.77km

Shortest route: 13.78 mins & 8.39km
